In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy
import emission.core.get_database as edb
import logging
import folium
import math

In [ ]:
import emission.storage.timeseries.abstract_timeseries as esta

In [ ]:
# copied from mode_purpose_share.ipynb
# We select participants from real data to be users at this time
participant_uuid_obj = list(edb.get_profile_db().find({"install_group": "participant"}, {"user_id": 1, "_id": 0}))
all_users = [u["user_id"] for u in participant_uuid_obj]

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
logging.debug('There are %s users, we will plot the graphs for one of them' % len(all_users))

user = all_users[1]

In [ ]:
import emission.analysis.modelling.tour_model.cluster_pipeline as eamtc

In [ ]:
# we should experiment with different values here
radius = 300

In [ ]:
import emission.analysis.modelling.tour_model.similarity as similarity

In [ ]:
import emission.analysis.modelling.tour_model.cluster_pipeline as pipeline

In [ ]:
trips = pipeline.read_data(uuid=user)

In [ ]:
bin_trips, bins = pipeline.remove_noise(trips, radius)

In [ ]:
bin_start_lat = []
bin_start_log = []

# Flip indices because points are in geojson (i.e. lon, lat)
for trip in bin_trips:
    trip_start_lat = trip.data.start_loc["coordinates"][1]
    trip_start_log = trip.data.start_loc["coordinates"][0]
    bin_start_lat.append(trip_start_lat)
    bin_start_log.append(trip_start_log)

In [ ]:
bin_end_lat = []
bin_end_log = []
for trip in bin_trips:
    trip_end_lat = trip.data.end_loc["coordinates"][1]
    trip_end_log = trip.data.end_loc["coordinates"][0]
    bin_end_lat.append(trip_end_lat)
    bin_end_log.append(trip_end_log)

In [ ]:
# bin_trips

In [ ]:
logging.debug('For the first trip, the start point is %s' % [bin_start_lat[0], bin_start_log[0]])

In [ ]:
logging.debug('For the first trip, the end point is %s' % [bin_end_lat[0],bin_end_log[0]])


In [ ]:
# Plot all the bin trips on the map, later we should use different color for different bins, but not sure whether it works.
m = folium.Map(location=[bin_start_lat[0], bin_start_log[0]],zoom_start=12, max_zoom= 30, control_scale=True)
for i in range (len(bin_trips)):
    layer = folium.PolyLine([[bin_start_lat[i],bin_start_log[i]],[bin_end_lat[i],bin_end_log[i]]],weight=2, color='orange')
    layer.add_to(m)
m

In [ ]:
########clusters#####

In [ ]:
# cluster the data using k-means
# def cluster(data, bins)
# return feat.clusters, feat.labels, feat.data
clusters,labels,feat_data = pipeline.cluster(bin_trips,len(bins))
clusters

In [ ]:
# Copy from featurization.py, should later change it to use folium
def map_clusters(self):
        import matplotlib.pyplot as plt
        from matplotlib import colors as matcol
        colormap = plt.cm.get_cmap()

        if self.labels:
            # mymap2 = pygmaps.maps(37.5, -122.32, 10)
            for i in range(len(self.points)):
                start_lat = self.points[i][1]
                start_lon = self.points[i][0]
                end_lat = self.points[i][3]
                end_lon = self.points[i][2]
                path = [(start_lat, start_lon), (end_lat, end_lon)]
                mymap2.addpath(path, matcol.rgb2hex(colormap(old_div(float(self.labels[i]),self.clusters))))
            mymap2.draw('./mylabels.html')